# Data Cleaning

In [1]:
import pandas as pd

## Daten laden

In [2]:
train_series = pd.read_parquet('../../data/raw/train_series.parquet')
train_events = pd.read_csv('../../data/raw/train_events.csv')

In [3]:
print(train_series.shape)
print(train_events.shape)

(127946340, 5)
(14508, 5)


## Serien ohne Events entfernen

In [4]:
series_id_list = train_series['series_id'].unique().tolist()

events_series_id_list = train_events.dropna()['series_id'].unique().tolist()

series_without_events = list(set(series_id_list) - set(events_series_id_list))

print('Serien ohne Events: \n', series_without_events)

Serien ohne Events: 
 ['c7b1283bb7eb', '0f9e60a8e56d', 'a3e59c2ce3f6', '390b487231ce', 'c5d08fc3e040', '2fc653ca75c7', '89c7daa72eee', 'e11b9d69f856']


In [5]:
cleaned_train_series = train_series[~train_series.series_id.isin(series_without_events)]
cleaned_train_events = train_events[~train_events.series_id.isin(series_without_events)]

## Gesäuberte Daten speichern

In [6]:
print(cleaned_train_series.shape)
print(cleaned_train_series.shape)

(124822080, 5)
(124822080, 5)


In [7]:
cleaned_train_series.to_parquet('../../data/cleaned/train_series.parquet')
cleaned_train_events.to_csv('../../data/cleaned/train_events.csv', index=False)